In [37]:
import folium

In [38]:
from folium import plugins

In [39]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [40]:
travel_index = pd.read_csv('hafas_vbb.csv', index_col=0)
travel_index

,latitude,longitude,min travel time,shortest trip
0,52.502793,13.473981,844.584445,Marktstr. (Berlin) > S Rummelsburg (Berlin) > ...
1,52.502793,13.467363,456.986839,S Ostkreuz Bhf (Berlin) > S+U Alexanderplatz B...
2,52.502793,13.460744,859.758908,Modersohnstr. (Berlin) > S+U Warschauer Str. (...
3,52.502793,13.454125,846.968438,Osthafen (Berlin) > S+U Warschauer Str. (Berli...
4,52.502793,13.447507,1039.271294,Oberbaumbrücke (Berlin) > S+U Warschauer Str. ...
5,52.508708,13.473981,1119.492022,Neue Bahnhofstr. (Berlin) > U Frankfurter Tor ...
6,52.508708,13.467363,747.374833,Boxhagener Str./Holteistr. (Berlin) > S+U Fran...
7,52.508708,13.460744,753.429644,Wühlischstr./Gärtnerstr. (Berlin) > S+U Frankf...
8,52.508708,13.454125,916.752917,Libauer Str. (Berlin) > Grünberger Str./Warsch...
9,52.508708,13.447507,859.535770,Helsingforser Platz (Berlin) > S Warschauer St...


In [41]:
m = folium.Map((52.515998, 13.457574), zoom_start=14)
for index, row in travel_index.iterrows():
    mtt = row['min travel time']
    if mtt < 600:
        color = 'green'
    elif mtt < 900:
        color = 'yellow'
    else:
        color = 'red'
    folium.CircleMarker((row['latitude'], row['longitude']), fill_color=color, opacity=1.0, color=color, stroke=True, tooltip='{:0.0f}s: {}'.format(mtt, row['shortest trip'])).add_to(m)

In [43]:
m